In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torchvision import transforms, models

from PIL import Image, ImageFile
import pickle

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os

In [2]:
def set_seed(seed):
    os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2"
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
table = "tabelaAlunosPPI_modificado.xls" # 201257335.jpg foi removida
df = pd.read_excel(table) 

In [5]:
# Manipulações com a tabela:
# Elimina colunas que não serão utilizadas
df = df.drop(columns=['idAluno', 'registroAcademico', 'curso',
       'campus', 'sigla', 'tipoIngresso', 'sistemaIngresso', 'parecerFase01',
       'parecerFase02', 'parecerFase03', 'parecerRecursal'])

# Elimina registros de 2018
df = df[df['anoIngresso'] != 2018]

# Cria uma nova coluna com o caminho da pasta do respectivo ano
caminhos = {
    2019: 'C:\\Users\\DELL\\Desktop\\Imagens2019\\fotos2019\\',
    2020: 'C:\\Users\\DELL\\Desktop\\Imagens2020\\fotos2020\\',
    2021: 'C:\\Users\\DELL\\Desktop\\Imagens2021\\fotos2021\\',
    2022: 'C:\\Users\\DELL\\Desktop\\Imagens2022\\fotos2022\\',
    2023: 'C:\\Users\\DELL\\Desktop\\Imagens2023\\fotos2023\\'
}
df['caminho'] = df['anoIngresso'].map(caminhos)

# Junta o caminho com o nome da imagem
df['caminho'] = df['caminho'] + df['foto']

# Elimina o restante das colunas que não serão mais utilizadas
df = df.drop(columns=['anoIngresso', 'foto'])

df = df.reset_index(drop=True)

In [6]:
# Separa o dataframe em dois: Um para brancos e outro para PPI
# Isso será importante para organizar os dados em train e test
df_branco = df[(df['parecerFinal'] == 'Não Consistente') | (df['parecerFinal'] == 'Não Consistente - Recurso')][['parecerFinal', 'caminho']]
df_PPI = df[df['parecerFinal'] == 'Consistente'][['parecerFinal', 'caminho']]

df_branco['parecerFinal'] = 0
df_PPI['parecerFinal'] = 1

df_branco = df_branco.reset_index(drop=True)
df_PPI = df_PPI.reset_index(drop=True)

In [7]:
df_branco.shape

(553, 2)

In [8]:
df_PPI.shape

(5237, 2)

In [9]:
class CustomDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    
    def __getitem__(self, index):
        image_path = self.data['caminho'][index]
        label = self.data['parecerFinal'][index]
        image = Image.open(image_path)
        image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return self.data.shape[0]

In [10]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=(-10,10)),
    transforms.ToTensor(),
    transforms.Normalize(mean=0.0009028730599446859, std=0.0004646305006696925)
])

In [11]:
num_epochs = 8
lr = 0.1
batch_size = 64

In [12]:
train_data_branco, test_data_branco = train_test_split(df_branco, test_size=0.1, random_state=42)
train_data_PPI, test_data_PPI = train_test_split(df_PPI, test_size=0.1, random_state=42)

train_data = pd.concat([train_data_branco, train_data_PPI], ignore_index=True)
test_data = pd.concat([test_data_branco, test_data_PPI], ignore_index=True)

In [13]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

custom_train = CustomDataset(train_data, transform=transform)
custom_test = CustomDataset(test_data, transform=transform)

In [14]:
train_loader = DataLoader(custom_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(custom_test, batch_size=batch_size, shuffle=True)

In [15]:
len(custom_train) + len(custom_test)

5790

In [16]:
weights = pickle.load(open('resnet50_scratch_weight.pkl', 'rb'))
model = models.resnet50(weights=weights)

c:\Users\DELL\anaconda3\envs\CUDA\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
for param in model.parameters():
    param.requires_grad = False

# for param in model.layer4.parameters():
#     param.requires_grad = True

model.fc = nn.Linear(2048, 2)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.2)

model.train()
for epoch in range(num_epochs):
    running_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output.squeeze(), labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    print(f'Epoch: {epoch + 1} Loss: {running_loss/len(train_loader)}')

Epoch: 1 Loss: 9.602381016423063
Epoch: 2 Loss: 2.1219516865363937
Epoch: 3 Loss: 4.187879474424735
Epoch: 4 Loss: 1.2777731865644455
Epoch: 5 Loss: 0.8483395934468363
Epoch: 6 Loss: 0.5796089219610866
Epoch: 7 Loss: 0.4587587557006173
Epoch: 8 Loss: 0.43135247443143915


In [19]:
# Teste

correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct // total} %')

Accuracy: 88 %


In [20]:
# Matriz de confusão

y_pred = []
y_true = []
classes = ['branco', 'PPI']

model.eval()
model.cpu()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted = predicted.cpu()
        y_pred.extend(predicted.numpy())
        y_true.extend(labels.numpy())

    y_pred = np.array(y_pred)
    y_true = np.array(y_true)

    cm = confusion_matrix(y_true, y_pred)

    print('\t\tPredicted Labels\n')

    print(f"True Labels Branco {cm[0]}")
    print(f"\t       PPI {cm[1]}") 
    print('\t\t   Branco PPI')

		Predicted Labels

True Labels Branco [14 42]
	       PPI [ 30 494]
		   Branco PPI
